# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']


J. Braspenning  ->  J. Braspenning  |  ['J. Braspenning']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 85 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2504.01076


extracting tarball to tmp_2504.01076...

 done.
Retrieving document from  https://arxiv.org/e-print/2504.01609


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2504.01076/erass1_x_hscsr.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'bcg_images' from 'tmp_2504.01076/bcg_images.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'pplots/posteriors' from 'tmp_2504.01076/pplots/posteriors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.

...

Found 86 bibliographic references in tmp_2504.01745/aanda.bbl.
Error retrieving bib data for migkas_cosmological_2021: 'author'
Error retrieving bib data for migkas_cosmological_2021: 'author'
Retrieving document from  https://arxiv.org/e-print/2504.01852


extracting tarball to tmp_2504.01852...

 done.


F. Walter  ->  F. Walter  |  ['F. Walter']


Found 266 bibliographic references in tmp_2504.01852/manuscript.bbl.
Error retrieving bib data for akins_cosmos-web_2024: 'author'
Error retrieving bib data for akins_cosmos-web_2024: 'author'
Error retrieving bib data for lin_discovery_2024: 'author'
Error retrieving bib data for noboriguchi_extreme_2022: 'author'
Error retrieving bib data for wang_rubies_2024: 'author'
Error retrieving bib data for mazzolari_new_2024: 'author'
Error retrieving bib data for harikane_jwstnirspec_2023: 'author'
Error retrieving bib data for  nakajima_jwst_2023: 'author'
Error retrieving bib data for  scholtz_jades_2023: 'author'
Error retrieving bib data for  ubler_ga-nifs_2024: 'author'
Error retrieving bib data for  juodzbalis_dormant_2024: 'author'
Error retrieving bib data for  rinaldi_not_2024: 'author'
Error retrieving bib data for rinaldi_not_2024: 'author'
Error retrieving bib data for backhaus_emission-line_2025: 'author'
Error retrieving bib data for matthee_deciphering_2022: 'author'
Error re

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01745-b31b1b.svg)](https://arxiv.org/abs/2504.01745) | **Characterising galaxy cluster scaling relations as cosmic isotropy tracers using the FLAMINGO simulations**  |
|| Y. He, et al. -- incl., <mark>J. Braspenning</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *17 pages, 7 figures, submitted to A&A*|
|**Abstract**|            The standard cosmological model, $\Lambda$CDM, assumes isotropy on large cosmic scales. However, recent studies using galaxy cluster scaling relations reported an apparent $H_0$ anisotropy at $5.4\sigma$ that could be attributed to large bulk flows extending beyond $500\,\mathrm{Mpc}$, in disagreement with $\Lambda$CDM. To quantify the statistical tension of the observational galaxy cluster data used in past studies with $\Lambda$CDM, we utilise the isotropic ($2.8\,\mathrm{Gpc})^3$ run of the FLAMINGO ($\Lambda$CDM) simulations, the largest hydrodynamical cosmological simulation available to date. We create 1728 simulated lightcones and study the apparent level of anisotropy traced by X-ray and thermal Sunyaev-Zeldovich scaling relations in the same cluster sample selection and methodology as in Migkas et al. (2021, arXiv:2103.13904). We find the probability of such apparent anisotropies randomly emerging in cluster scaling relations within a $\Lambda$CDM universe to be $0.12\%\, (3.2\sigma)$. The discrepancy goes up to $\sim 3.6\sigma$ when modelled as a bulk flow at $z < 0.1$. We find that statistical noise accounts for over $80\%$ of the anisotropy amplitude in each lightcone, with large peculiar velocities contributing less than $20\%$. We also show that anisotropy amplitudes are highly sensitive to the intrinsic scatter in the scaling relations, with tighter relations providing stronger constraints. Nevertheless, the tension between Migkas et al. (2021, arXiv:2103.13904) and $\Lambda$CDM persists, however, at a lower significance than previously reported.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01852-b31b1b.svg)](https://arxiv.org/abs/2504.01852) | **Deciphering the Nature of Virgil: An Obscured AGN Lurking Within an Apparently Normal Lyman-α Emitter During Cosmic Reionization**  |
|| P. Rinaldi, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *39 pages, 3 tables, 14 figures. Submitted to ApJ*|
|**Abstract**|            We present a comprehensive analysis of the MIRI Extremely Red Object Virgil, a Lyman-$\alpha$ emitter at $z_{spec} = 6.6379 \pm 0.0035$ with the photometric properties of a Little Red Dot. Leveraging new JWST/MIRI imaging from the MIDIS and PAHSPECS programs, we confirm Virgil's extraordinary nature among galaxies in JADES/GOODS-South, exhibiting a strikingly red NIRCam-to-MIRI color (F444W $-$ F1500W = $2.84\pm0.04$~mag). Deep NIRSpec/PRISM spectroscopy from the OASIS program offers key insights into the host galaxy, revealing properties of an average star-forming galaxy during Cosmic Reionization, such as a subsolar metallicity, low-to-moderate dust content, and a relatively high ionization parameter and electron temperature. By estimating the star formation rate of Virgil from UV and H$\alpha$, we find evidence that the galaxy is either entering or fading out of a bursty episode. Although line-ratio diagnostics employed at high-$z$ would classify Virgil as an Active Galactic Nucleus (AGN), this classification becomes ambiguous once redshift evolution is considered. Nonetheless, Virgil occupies the same parameter space as recently confirmed AGNs at similar redshifts. The new deep MIRI data at 15~$\mu$m reinforce the AGN nature of Virgil, as inferred from multiple spectral energy distribution (SED) fitting codes. Virgil's rising infrared SED and UV excess resemble those of Dust-Obscured Galaxies (DOGs) studied with Spitzer at Cosmic Noon, particularly blue-excess HotDOGs. Our results highlight the need for a multi-wavelength approach incorporating MIRI to uncover such extreme sources at $z\gtrsim6$ and to shed light on the interplay between galaxy evolution and early black hole growth during Cosmic Reionization.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01076-b31b1b.svg)](https://arxiv.org/abs/2504.01076) | **The SRG/eROSITA All-Sky Survey. The Weak-Lensing Mass Calibration and the Stellar Mass-to-Halo Mass Relation from the Hyper Suprime-Cam Subaru Strategic Program**  |
|| I.-N. Chiu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *26 pages, 10 figures. Submitted to A&A*|
|**Abstract**|            We present the weak-lensing mass calibration of 124 galaxy clusters and groups at redshift $0.1<z<0.8$ in the first Data Release of the $eROSITA$ All-Sky Survey ($eRASS1$) and constrain their BCG stellar-mass-to-halo-mass-and-redshift relation, using the data sets from the Hyper Suprime-Cam (HSC) Subaru Strategic Program. The cluster survey is conducted by the $eROSITA$ X-ray telescope aboard the Spectrum-Roentgen-Gamma space observatory. The cluster sample is X-ray-selected and optically confirmed with a negligibly low contamination rate. On a basis of individual clusters, the shear profiles of $96$ clusters are derived using the HSC Three-Year weak-lensing data, while the BCG stellar masses $M_{\star,\mathrm{BCG}}$ of $101$ clusters are estimated using the SED template fitting to the HSC five-band photometry. The count rate ($C_{\mathrm{R}}-M-z$) and BCG stellar mass ($M_{\star,\mathrm{BCG}}-M-z$) relations are simultaneously constrained in a population modelling. The $C_{\mathrm{R}}-M-z$ relation reveals a mass trend of $\propto M^{1.50^{+0.20}_{-0.30}}$, which is steeper than the self-similar prediction, and no deviation from the self-similar redshift scaling. We obtain the power-law indices of the mass ($B_{\mathrm{BCG}} = 0.25^{+0.11}_{-0.15}$) and redshift ($\gamma_{\mathrm{BCG}} = 0.87^{+0.92}_{-0.78}$) scaling for the $M_{\star,\mathrm{BCG}}-M-z$ relation, and find a strong degeneracy between them. By adopting an informative prior on $\gamma_{\mathrm{BCG}}$ to break the degeneracy, we obtain a statistically consistent $M_{\star,\mathrm{BCG}}-M-z$ relation. Our results suggest that the BCG stellar mass at a fixed halo mass has remained stable with a moderate increase since redshift $z\approx0.8$. This finding supports the picture of the ``rapid-then-slow'' BCG formation, where the majority of the stellar mass must have been assembled at much earlier cosmic time.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01613-b31b1b.svg)](https://arxiv.org/abs/2504.01613) | **The Mini-SiTian Array: Design and application of Master Control System**  |
|| Z. Wang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *16 pages, 7 figures, Accepted for publication in a special issue of Research in Astronomy and Astrophysics on the Mini-SiTian Array*|
|**Abstract**|            The SiTian Project represents a groundbreaking initiative in astronomy, aiming to deploy a global network of telescopes, each with a 1-meter aperture, for comprehensive time-domain sky surveys. The network's innovative architecture features multiple observational nodes, each comprising three strategically aligned telescopes equipped with filters. This design enables three-color (g, r, i) channel imaging within each node, facilitating precise and coordinated observations. As a pathfinder to the full-scale project, the Mini-SiTian Project serves as the scientific and technological validation platform, utilizing three 30-centimeter aperture telescopes to validate the methodologies and technologies planned for the broader SiTian network. This paper focuses on the development and implementation of the Master Control System (MCS),and the central command hub for the Mini-SiTian array. The MCS is designed to facilitate seamless communication with the SiTian Brain, the project's central processing and decision-making unit, while ensuring accurate task allocation, real-time status monitoring, and optimized observational workflows. The system adopts a robust architecture that separates front-end and back-end functionalities.A key innovation of the MCS is its ability to dynamically adjust observation plans in response to transient source alerts, enabling rapid and coordinated scans of target sky regions...(abridged)         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01614-b31b1b.svg)](https://arxiv.org/abs/2504.01614) | **Searching for kilonova with the SiTian prototype telescope**  |
|| Z. Li, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *16 pages, 7 figures. Accepted for publication in a special issue of Research in Astronomy and Astrophysics on the Mini-SiTian Array*|
|**Abstract**|            We simulate the optical searching of gravitational-wave electromagnetic counterpart of the binary neutron star (BNS) merger event (i.e., a kilonova) using the ground based {\it SiTian} project prototype telescope with a 5-min limiting magnitude of 22.0, triggered by LIGO and Virgo gravitational wave detectors during the ongoing O4 run. Our simulations show that an average of 0.17-0.25 kilonova events can be observed over the entire O4 period of $\sim 2$ years in the most optimistic case we set, while no kilonova can be detected in other cases. We note that it is beneficial for {\it SiTian}'s kilonova searching by extending the exposure time to gain deeper limiting magnitude despite the rapid decline of kilonova luminosity.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01615-b31b1b.svg)](https://arxiv.org/abs/2504.01615) | **The Mini-SiTian Array: A Pathfinder for the SiTian Project**  |
|| Y. Huang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *4 pages, Accepted for publication in a special issue of Research in Astronomy and Astrophysics on the Mini-SiTian Array*|
|**Abstract**|            The Mini-SiTian Array serves as a pathfinder for the SiTian project, which aims to survey the entire sky in $gri$ bands every 30 minutes, reaching a limiting magnitude of 21. This special issue features 11 papers covering the design, operation, data reduction, and early scientific results from two years of Mini-SiTian observations. The insights gained from these pathfinder experiments represent a significant milestone toward the full realization of the SiTian project.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01616-b31b1b.svg)](https://arxiv.org/abs/2504.01616) | **The Mini-SiTian Array: Imaging Processing Pipeline**  |
|| K. Xiao, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-04-03*|
|*Comments*| *24 pages, 17 figures, 1 table, accepted for publication in a special issue of Research in Astronomy and Astrophysics on the Mini-SiTian Array, see main results in Figures 11, 12, and 15*|
|**Abstract**|            As a pathfinder of the SiTian project, the Mini-SiTian (MST) array, employed three commercial CMOS cameras, represents a next-generation, cost-effective optical time-domain survey project. This paper focuses primarily on the precise data processing pipeline designed for wide-field, CMOS-based devices, including the removal of instrumental effects, astrometry, photometry, and flux calibration. When applying this pipeline to approximately 3000 observations taken in the Field 02 (f02) region by MST, the results demonstrate a remarkable astrometric precision of approximately 70--80\,mas (about 0.1\,pixel), an impressive calibration accuracy of approximately 1\,mmag in the MST zero points, and a photometric accuracy of about 4\,mmag for bright stars. Our studies demonstrate that MST CMOS can achieve photometric accuracy comparable to that of CCDs, highlighting the feasibility of large-scale CMOS-based optical time-domain surveys and their potential applications for cost optimization in future large-scale time-domain surveys, like the SiTian project.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2504.01629-b31b1b.svg)](https://arxiv.org/abs/2504.01629) | **A 0.3\% calibration of the W UMa-type contact binary luminosity based on Gaia DR3**  |
|| <mark>J. Li</mark>, et al. |
|*Appeared on*| *2025-04-03*|
|*Comments*| *11 pages, 10 figures, accepted in MNRAS*|
|**Abstract**|            W Ursa Majoris (W UMa)-type contact binary systems (CBs) with period--luminosity (PL) relations are valuable distance indicators. The PL relations of CBs are affected by metallicity. Here, we establish PL relations and period--luminosity--metallicity (PLZ) relations in nine bands from optical to mid-infrared ($BP$, $G$, $RP$, $J$, $H$, $K_S$, $W1$, $W2$, $W3$) and in five Wesenheit bands based on Gaia DR3 parallaxes. The dispersion of PLZ relations gradually decreases from the optical to mid-infrared bands, with the minimum dispersion of 0.138 mag. We fit the best PL relations for three bands ($W1$, $W_{G,BP,RP}$, $W_{W1,BP,RP}$) under different parallax uncertainty criteria and determine a parallax (after correction) zero point of $zp_\varpi=24\pm4$ $\mu$as. After fixing the parallax zero point, we find that the total zero errors of the PL and PLZ relation are smallest when the parallax uncertainty is less than 2\%, resulting in a calibrated CB luminosity with an accuracy of 0.33\%, which is more accurate than the classical Cepheids. Furthermore, by examining the absolute magnitude residuals in different metallicity intervals, we find that the use of a linear metallicity effect is appropriate for CBs with different metallicities. These results indicate that CBs are excellent standard candles.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2504.01745/./figures/evs_joint.png', 'tmp_2504.01745/./figures/evs_joint_m21.png', 'tmp_2504.01745/./figures/evs_joint_scatter.png', 'tmp_2504.01745/./figures/evs_joint_m21_scatter.png', 'tmp_2504.01745/./figures/h0mc_hist_joint_combined.png', 'tmp_2504.01745/./figures/h0_scan_hist_LTYTMT.png', 'tmp_2504.01745/./figures/h0mc_hist_LTYTMT.png', 'tmp_2504.01745/./figures/h0_scan_hist_LTYTMT_scatter.png', 'tmp_2504.01745/./figures/h0mc_hist_LTYTMT_scatter.png']
copying  tmp_2504.01745/./figures/evs_joint.png to _build/html/
copying  tmp_2504.01745/./figures/evs_joint_m21.png to _build/html/
copying  tmp_2504.01745/./figures/evs_joint_scatter.png to _build/html/
copying  tmp_2504.01745/./figures/evs_joint_m21_scatter.png to _build/html/
copying  tmp_2504.01745/./figures/h0mc_hist_joint_combined.png to _build/html/
copying  tmp_2504.01745/./figures/h0_scan_hist_LTYTMT.png to _build/html/
copying  tmp_2504.01745/./figures/h0mc_hist_LTYTMT.png to _build/html/
copying  tmp_2

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\dash}{\text{--}}$
$\newcommand{\LT}{L_\mathrm{X}\dash T}$
$\newcommand{\YT}{Y_\mathrm{SZ}\dash T}$
$\newcommand{\MT}{M_\mathrm{gas}\dash T}$
$\newcommand{\placehold}[1]{{\color{green}\texttt{[{#1}]}}}$</div>



<div id="title">

# Characterising galaxy cluster scaling relations as cosmic isotropy tracers using the FLAMINGO simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.01745-b31b1b.svg)](https://arxiv.org/abs/2504.01745)<mark>Appeared on: 2025-04-03</mark> -  _17 pages, 7 figures, submitted to A&A_

</div>
<div id="authors">

Y. He, et al. -- incl., <mark>J. Braspenning</mark>

</div>
<div id="abstract">

**Abstract:** The standard cosmological model, $\Lambda$ CDM, assumes isotropy on large cosmic scales. However, recent studies using galaxy cluster scaling relations reported an apparent $H_0$ anisotropy at $5.4\sigma$ that could be attributed to large bulk flows extending beyond $\SI{500}{Mpc}$ , in disagreement with $\Lambda$ CDM. To quantify the statistical tension of the observational galaxy cluster data used in past studies with $\Lambda$ CDM, we utilize the isotropic ( $\SI{2.8}{Gpc})^3$ run of the FLAMINGO ( $\Lambda$ CDM) simulations, the largest hydrodynamical cosmological simulation available to date. We create 1728 simulated lightcones and study the apparent level of anisotropy traced by X-ray and thermal Sunyaev-Zeldovich scaling relations in the same cluster sample selection and methodology as in migkas_cosmological_2021 . We find the probability of such apparent anisotropies randomly emerging in cluster scaling relations within a $\Lambda$ CDM universe to be $0.12\%  (3.2\sigma)$ . The discrepancy goes up to $\sim 3.6\sigma$ when modelled as a bulk flow at $z < 0.1$ . We find that statistical noise accounts for over $80\%$ of the anisotropy amplitude in each lightcone, with large peculiar velocities contributing less than $20\%$ . We also show that anisotropy amplitudes are highly sensitive to the intrinsic scatter in the scaling relations, with tighter relations providing stronger constraints. Nevertheless, the tension between migkas_cosmological_2021 and $\Lambda$ CDM persists, however, at a lower significance than previously reported.

</div>

<div id="div_fig1">

<img src="tmp_2504.01745/./figures/evs_joint.png" alt="Fig10.1" width="25%"/><img src="tmp_2504.01745/./figures/evs_joint_m21.png" alt="Fig10.2" width="25%"/><img src="tmp_2504.01745/./figures/evs_joint_scatter.png" alt="Fig10.3" width="25%"/><img src="tmp_2504.01745/./figures/evs_joint_m21_scatter.png" alt="Fig10.4" width="25%"/>

**Figure 10. -** EVS analyses of joint $H_0$ anisotropy in the FLAMINGO lightcones using the MCMC (left) and M21 (right) methods, without (top) and with (bottom) matched scatter. Grey and orange histograms show the lower-bound projection of all 1728 lightcones without and with scatter, respectively. The red vertical line marks the M21 result under the same projection. The blue dashed line indicates the GPD threshold (90th or 95th percentile), above which the tail is modelled by a Generalized Pareto Distribution (GPD), shown in green. Annotated in red is the EVS probability of obtaining the M21 result in FLAMINGO. (*fig:evs-all-joint*)

</div>
<div id="div_fig2">

<img src="tmp_2504.01745/./figures/h0mc_hist_joint_combined.png" alt="Fig3" width="100%"/>

**Figure 3. -** Joint MCMC constraint from the $\LT$ and $\YT$ relations, compared to the combined M21 result (red dot). The orange contour includes injected scatter, while the grey does not. Contours show the 39\%, 86\%, and 98.9\% confidence regions. (*fig:h0-result-joint-MCMC*)

</div>
<div id="div_fig3">

<img src="tmp_2504.01745/./figures/h0_scan_hist_LTYTMT.png" alt="Fig9.1" width="25%"/><img src="tmp_2504.01745/./figures/h0mc_hist_LTYTMT.png" alt="Fig9.2" width="25%"/><img src="tmp_2504.01745/./figures/h0_scan_hist_LTYTMT_scatter.png" alt="Fig9.3" width="25%"/><img src="tmp_2504.01745/./figures/h0mc_hist_LTYTMT_scatter.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Distribution of $H_0$ variation and its statistical significance from 1728 lightcones using the M21 scanning method (left) and the MCMC method (right), with (bottom) and without (top) mock scatter. Blue solid, red dashed, and green dash-dotted contours correspond to constraints from the $\LT$, $\YT$, and $\MT$ relations, respectively. Contour levels indicate 39\%, 86\%, and 98.9\% probability regions (equivalent to 1, 2, and 3$\sigma$ in two dimensions). Blue (red) squares mark the M21 results from $\LT$($\YT$), including their measured statistical significance. Blue (red) circles show the M21 results obtained from isotropic Monte Carlo realisations for $\LT$($\YT$). (*fig:h0-result*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.01745"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\gsim}{{\;\raise0.3ex\hbox{>\kern-0.75em\raise-1.1ex\hbox{\sim}}\;}}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# $\bf$ Deciphering the Nature of $_ Virgil_$: An Obscured AGN Lurking Within an Apparently Normal Lyman-$\alpha$ Emitter During Cosmic Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2504.01852-b31b1b.svg)](https://arxiv.org/abs/2504.01852)<mark>Appeared on: 2025-04-03</mark> -  _39 pages, 3 tables, 14 figures. Submitted to ApJ_

</div>
<div id="authors">

P. Rinaldi, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a comprehensive analysis of the MIRI Extremely Red Object _Virgil_ , a Lyman- $\alpha$ emitter at $z_{spec} = 6.6379 \pm0.0035$ with the photometric properties of a Little Red Dot. Leveraging new JWST/MIRI imaging from the MIDIS and PAHSPECS programs, we confirm _Virgil_ 's extraordinary nature among galaxies in JADES/GOODS-South, exhibiting a strikingly red NIRCam-to-MIRI color (F444W $-$ F1500W = $2.84\pm0.04$ mag). Deep NIRSpec/PRISM spectroscopy from the OASIS program offers key insights into the host galaxy, revealing properties of an average star-forming galaxy during Cosmic Reionization, such as a subsolar metallicity, low-to-moderate dust content, and a relatively high ionization parameter and electron temperature. By estimating the star formation rate of _Virgil_ from UV and H $\alpha$ , we find evidence that the galaxy is either entering or fading out of a bursty episode. Although line-ratio diagnostics employed at high- $z$ would classify _Virgil_ as an Active Galactic Nucleus (AGN), this classification becomes ambiguous once redshift evolution is considered. Nonetheless, _Virgil_ occupies the same parameter space as recently confirmed AGNs at similar redshifts. The new deep MIRI data at 15 $\mu$ m reinforce the AGN nature of _Virgil_ , as inferred from multiple spectral energy distribution (SED) fitting codes. _Virgil_ 's rising infrared SED and UV excess resemble those of Dust-Obscured Galaxies (DOGs) studied with $_ Spitzer_$ at Cosmic Noon, particularly blue-excess HotDOGs. Our results highlight the need for a multi-wavelength approach incorporating MIRI to uncover such extreme sources at $z\gtrsim6$ and to shed light on the interplay between galaxy evolution and early black hole growth during Cosmic Reionization.

</div>

<div id="div_fig1">

<img src="tmp_2504.01852/./Virgil_SED.png" alt="Fig11" width="100%"/>

**Figure 11. -** The best-fit results for _ Virgil_ are shown, with photometric data indicated by black squares and $3\sigma$ upper limits by black inverted triangles. The reduced chi-squared values are also reported and computed consistently across the different SED fitting codes to account for their varying treatments of upper limits. The red arrow highlights the IR excess relative to the typical SFG template, which both NIRCam and NIRSpec data would bias the SED fitting toward. In contrast, the blue arrow indicates the UV excess compared to the average LRD template proposed by akins_cosmos-web_2024.
    For comparison, we include the LRD model from akins_cosmos-web_2024 and _ Virgil_'s SED, excluding the MIRI bands, as derived from {\sc bagpipes}. Additionally, we show the average BL GP SED (green circles) from lin_discovery_2024, considering only sources with $M_{\bullet}$ similar to the estimated value for _ Virgil_. Finally, we compare it with the Blue-excess HotDOG (HSC J1202; red circles) from noboriguchi_extreme_2022. RUBIES-BLAGN-1 model from wang_rubies_2024 is shown in blue. The vertical dashed red line marks the transition between the NIRCam and MIRI domains. All models are normalized to F444W in the _ Virgil_'s reference system.
 (*fig:sed_fitting*)

</div>
<div id="div_fig2">

<img src="tmp_2504.01852/./Virgil_line_diagnostic_Mazzolari.png" alt="Fig9" width="100%"/>

**Figure 9. -** ** Left Panel:** O32 vs. O3Hg. ** Middle Panel:** Ne3O2 vs. O3Hg. ** Right Panel:** O33 vs. O3Hg. _ Virgil_ is shown as a red filled circle, with higher opacity representing the case without dust correction and lower opacity indicating the dust-corrected one. The dashed black lines indicate the separation criteria proposed by mazzolari_new_2024 to distinguish AGNs from SFGs. The contour areas correspond to the SDSS sample in the local Universe ( (abazajian_seventh_2009) ) for comparison. Filled circles represent pre-JWST literature data for SFGs and AGNs  (mostly at low redshift;  (seyfert_nuclear_1943,  izotov_chemical_2006,  berg_direct_2012,   amorin_extreme_2015,  perna_x-raysdss_2017,  yang_blueberry_2017,  yang_ly_2017,  izotov_j08114730_2018,  dors_chemical_2020,  armah_chemical_2021,   pustilnik_xmp_2021) ). For comparison with recent JWST-based studies, we include results from harikane_jwstnirspec_2023,  nakajima_jwst_2023,  scholtz_jades_2023,  ubler_ga-nifs_2024,  juodzbalis_dormant_2024,  rinaldi_not_2024(with some of them having both SFGs and AGNs). In particular, the sample from rinaldi_not_2024 consists of photometrically selected LRDs in GOODS-S, some exhibiting clear broadening in the Balmer lines. The gray dashed line in the middle panel represents the separation criterion at $z\gtrsim4$ recently proposed by backhaus_emission-line_2025 to distinguish BLAGNs from non-BLAGNs.
 (*fig:virgil_line_ratios_oiii_hg*)

</div>
<div id="div_fig3">

<img src="tmp_2504.01852/./Virgil_spectrum.png" alt="Fig4" width="100%"/>

**Figure 4. -** ** Left panel:** 2D MSA/PRISM spectrum produced by {\sc MSAEXP}. We optimally scaled the trace to highlight all of the significant line detections. We show the data in azure, while the uncertainty is highlighted in gray. Assuming the best-fit {\sc MSAEXP}$z_{spec} = 6.6379\pm0.0035$, we show the positions of the emission lines as dashed vertical lines. ** Right panel:** JWST/NIRCam RGB image (2.5$\arcsec$$\times$2.5$\arcsec$) along with the slit positions showed in white. We highlight _ Virgil_ and the foreground LAE at $z\approx4.77$ already studied in matthee_deciphering_2022 with VLT/MUSE data. (*fig:virgil_spectrum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2504.01852"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

421  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
